The images in the dataset were all different sizes. It was important to normalize them all to a desired dimension as well as omit anything that was below the desired dimension.

In [28]:
import glob
import cv2

desired_dimension = 512
skipped_images = 0
kept_images = 0

for filepath in glob.iglob('images/*.jpg'):
    img = cv2.imread(filepath)
    if (img.shape[0] >= desired_dimension and img.shape[1] >= desired_dimension):
        kept_images += 1
        res_img = cv2.resize(img, dsize=(desired_dimension, desired_dimension), interpolation=cv2.INTER_AREA)
        cv2.imwrite(f'preprocessed_images/{str(kept_images)}.jpg', res_img)
    else:
        skipped_images +=1

print('Kept Images: ' + str(kept_images))
print('Skipped Images: ' + str(skipped_images))

Kept Images: 3875
Skipped Images: 444


In [5]:
import cv2
import numpy as np
from skimage.util import random_noise
from keras import Model, Input
from keras.layers import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten
import tensorflow as tf
import time

Functions for working with, processing, and showing the image data

In [14]:
def get_dataset(num_samples, train_size, test_size, noise_amount, orig_size, low_size, color_channels):
    # Initialize np array first, then reassign for better performance
    x = np.empty([num_samples, low_size, low_size, color_channels])
    y = np.empty([num_samples, orig_size, orig_size, color_channels])
    opencv = np.empty([num_samples, orig_size, orig_size, color_channels])
    for i in range(1, num_samples+1):
        img_orig = cv2.imread(f'preprocessed_images/{str(i)}.jpg')
        img_low_res = cv2.resize(img_orig, dsize=(low_size, low_size), interpolation=cv2.INTER_AREA)
        img_low_res_noise = random_noise(img_low_res, mode='s&p', amount=noise_amount)
        img_low_res_upscaled_test = cv2.resize(img_low_res_noise, dsize=(orig_size, orig_size), interpolation=cv2.INTER_AREA)

        # Adding the noise resizes the rgb values to 0-1
        x[i-1] = img_low_res_noise
        # But the orinal image is still 0-255
        y[i-1] = img_orig / 255.0
        opencv[i-1] = img_low_res_upscaled_test
    
    x_test = x[-test_size:]
    y_test = y[-test_size:]
    x_train = x[:train_size]
    y_train = y[:train_size]
    opencv_test = opencv[-test_size:]
    opencv_train = opencv[:train_size]
        
    return x_train, y_train, x_test, y_test, opencv_test, opencv_train

def show_images(x_rgb, y_rgb, out_rgb, opencv_rgb, index, test_or_train):
    cv2.imshow(test_or_train + ' Image Before: ' + str(index), x_rgb[index])
    cv2.imshow(test_or_train + ' Image Desired: ' + str(index), y_rgb[index])
    cv2.imshow(test_or_train + ' Image Predicted: ' + str(index), out_rgb[index])
    cv2.imshow(test_or_train + ' OpenCV Prediction With Noise: ' + str(index), opencv_rgb[index])

def get_mse(x, y):
    a = x.flatten()
    b = y.flatten()
    return (np.square(a - b).mean())

The two models to initially test

In [7]:
def get_model_convolutions(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return Model(inputs, outputs)

def get_model_max_pooling(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return Model(inputs, outputs)

Running the models - changing parameters

In [11]:
def run_model(model, batch_size, epochs, optimizer, x_train, y_train, x_test, y_test, opencv_test, opencv_train):
    start = time.time()

    # Finish model
    model.compile(optimizer=optimizer, loss='mse')
    # Train the neural network
    model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs)
    
    end = time.time()
    
    # Process model out back to np.uint8 type - round to nearest ints
    out_train = model.predict(x_train) * 255.0
    out_rgb_train = out_train.clip(0, 255).astype(np.uint8)
    y_rgb_train = np.rint((y_train * 255.0)).astype(np.uint8)
    x_rgb_train = np.rint((x_train * 255.0)).astype(np.uint8)

    out_test = model.predict(x_test) * 255.0
    out_rgb_test = out_test.clip(0, 255).astype(np.uint8)
    y_rgb_test = np.rint((y_test * 255.0)).astype(np.uint8)
    x_rgb_test = np.rint((x_test * 255.0)).astype(np.uint8)
    
    opencv_rgb_test = np.rint((opencv_test * 255.0)).astype(np.uint8)
    opencv_rgb_train = np.rint((opencv_train * 255.0)).astype(np.uint8)

    # Get metrics from model
    print('MSE of Model (Train): ' + str(get_mse(y_rgb_train, out_rgb_train)))
    print('MSE of Model (Test): ' + str(get_mse(y_rgb_test, out_rgb_test)))
    print('Runtime (seconds): ' + str(end - start))
    # print('MSE of OpenCV Simple Resize: ' + str(get_mse(y_rgb, test_opencv_rgb)))

    # Show Example Images
    show_images(x_rgb_train, y_rgb_train, out_rgb_train, opencv_rgb_train, 0, 'Training')
    show_images(x_rgb_test, y_rgb_test, out_rgb_test, opencv_rgb_test, 20, 'Testing')
    show_images(x_rgb_test, y_rgb_test, out_rgb_test, opencv_rgb_test, 39, 'Testing')
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Image params
noise_amount = 0.0005
upscale_factor = 2
orig_size = 512
low_size = 256
color_channels = 3
# Model params
num_samples = 200
batch_size = 20
train_size = 160
test_size = 40
epochs = 300
optimzer = 'rmsprop'

x_train, y_train, x_test, y_test, opencv_test, opencv_train = get_dataset(num_samples, train_size, test_size, noise_amount, orig_size, low_size, color_channels)


In [16]:
model_convolutions = get_model_convolutions(upscale_factor, color_channels)
run_model(model_convolutions, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)


8/8 [==============================] - 1s 147ms/step - loss: 0.0066
Epoch 68/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0047
Epoch 69/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0085
Epoch 70/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0062
Epoch 71/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0080
Epoch 72/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0047
Epoch 73/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0058
Epoch 74/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0081
Epoch 75/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0071
Epoch 76/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0076
Epoch 77/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0034
Epoch 78/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0119
Epoch 79/300
8/8 [======================

In [19]:
model_pooling = get_model_max_pooling(upscale_factor, color_channels)
run_model(model_pooling, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)

092
Epoch 67/300
8/8 [==============================] - 1s 124ms/step - loss: 0.0089
Epoch 68/300
8/8 [==============================] - 1s 125ms/step - loss: 0.0071
Epoch 69/300
8/8 [==============================] - 1s 124ms/step - loss: 0.0093
Epoch 70/300
8/8 [==============================] - 1s 124ms/step - loss: 0.0074
Epoch 71/300
8/8 [==============================] - 1s 90ms/step - loss: 0.0088
Epoch 72/300
8/8 [==============================] - 1s 104ms/step - loss: 0.0064
Epoch 73/300
8/8 [==============================] - 1s 92ms/step - loss: 0.0110
Epoch 74/300
8/8 [==============================] - 1s 92ms/step - loss: 0.0087
Epoch 75/300
8/8 [==============================] - 1s 93ms/step - loss: 0.0074
Epoch 76/300
8/8 [==============================] - 1s 93ms/step - loss: 0.0084
Epoch 77/300
8/8 [==============================] - 1s 106ms/step - loss: 0.0068
Epoch 78/300
8/8 [==============================] - 1s 103ms/step - loss: 0.0095
Epoch 79/300
8/8 [===========

In [20]:
optimizer = 'adam'

model_base = get_model_base(upscale_factor, color_channels)
run_model(model_base, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)

00
8/8 [==============================] - 1s 145ms/step - loss: 0.0046
Epoch 68/300
8/8 [==============================] - 1s 144ms/step - loss: 0.0085
Epoch 69/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0073
Epoch 70/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0070
Epoch 71/300
8/8 [==============================] - 1s 145ms/step - loss: 0.0064
Epoch 72/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0054
Epoch 73/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0081
Epoch 74/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0064
Epoch 75/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0053
Epoch 76/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0065
Epoch 77/300
8/8 [==============================] - 1s 147ms/step - loss: 0.0036
Epoch 78/300
8/8 [==============================] - 1s 146ms/step - loss: 0.0107
Epoch 79/300
8/8 [====================

In [21]:
optimizer = 'rmsprop'
epochs = 100

model_base = get_model_base(upscale_factor, color_channels)
run_model(model_base, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)

Epoch 1/100
8/8 [==============================] - 2s 147ms/step - loss: 0.1936
Epoch 2/100
8/8 [==============================] - 1s 147ms/step - loss: 0.0631
Epoch 3/100
8/8 [==============================] - 1s 147ms/step - loss: 0.0384
Epoch 4/100
8/8 [==============================] - 1s 149ms/step - loss: 0.0320
Epoch 5/100
8/8 [==============================] - 1s 149ms/step - loss: 0.0194
Epoch 6/100
8/8 [==============================] - 1s 148ms/step - loss: 0.0193
Epoch 7/100
8/8 [==============================] - 1s 147ms/step - loss: 0.0248
Epoch 8/100
8/8 [==============================] - 1s 139ms/step - loss: 0.0223
Epoch 9/100
8/8 [==============================] - 1s 111ms/step - loss: 0.0207
Epoch 10/100
8/8 [==============================] - 1s 111ms/step - loss: 0.0085
Epoch 11/100
8/8 [==============================] - 1s 123ms/step - loss: 0.0309
Epoch 12/100
8/8 [==============================] - 1s 153ms/step - loss: 0.0216
Epoch 13/100
8/8 [===================

In [26]:
optimizer = 'rmsprop'
epochs = 500

model_base = get_model_base(upscale_factor, color_channels)
run_model(model_base, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)

====] - 1s 123ms/step - loss: 0.0035
Epoch 268/500
8/8 [==============================] - 1s 147ms/step - loss: 0.0034
Epoch 269/500
8/8 [==============================] - 1s 141ms/step - loss: 0.0036
Epoch 270/500
8/8 [==============================] - 1s 122ms/step - loss: 0.0033
Epoch 271/500
8/8 [==============================] - 1s 129ms/step - loss: 0.0036
Epoch 272/500
8/8 [==============================] - 1s 140ms/step - loss: 0.0034
Epoch 273/500
8/8 [==============================] - 1s 124ms/step - loss: 0.0034
Epoch 274/500
8/8 [==============================] - 1s 121ms/step - loss: 0.0030
Epoch 275/500
8/8 [==============================] - 1s 125ms/step - loss: 0.0044
Epoch 276/500
8/8 [==============================] - 1s 113ms/step - loss: 0.0033
Epoch 277/500
8/8 [==============================] - 1s 112ms/step - loss: 0.0027
Epoch 278/500
8/8 [==============================] - 1s 113ms/step - loss: 0.0040
Epoch 279/500
8/8 [==============================] - 1s 113ms

In [24]:
optimizer = 'rmsprop'
epochs = 5

model_base = get_model_base(upscale_factor, color_channels)
run_model(model_base, batch_size, epochs, optimzer, x_train, y_train, x_test, y_test, opencv_test, opencv_train)

Epoch 1/5
8/8 [==============================] - 2s 146ms/step - loss: 0.2207
Epoch 2/5
8/8 [==============================] - 1s 149ms/step - loss: 0.0425
Epoch 3/5
8/8 [==============================] - 1s 139ms/step - loss: 0.0342
Epoch 4/5
8/8 [==============================] - 1s 140ms/step - loss: 0.0684
Epoch 5/5
8/8 [==============================] - 1s 139ms/step - loss: 0.0366
MSE of Model (Train): 106.66216359138488
MSE of Model (Test): 107.06289383570353
Runtime (seconds): 6.4979023933410645
